In [1]:
import os, re, bz2, time, unicodedata
import gc
from tqdm import tqdm
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

In [2]:
wiki_xml = "data/enwiki.xml.bz2"
dino_file = "data/dino_names.txt"
output_dir = "data/output_clean"

In [3]:
spark = (
    SparkSession.builder
    .appName("WikiDinosaurPageExtractor")
    .master("local[*]")
    .config("spark.driver.memory", "12g")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

In [4]:
spark

In [5]:
def normalize_text(text):
    text = text.lower().replace(" ", "-")
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )

with open(dino_file, "r", encoding="utf-8") as f:
    dino_names = [normalize_text(line.strip()) for line in f if line.strip()]

dino_set = set(dino_names)
print(f"Loaded {len(dino_set)} dinosaur names")

Loaded 6958 dinosaur names


In [6]:
def group_pages(iterator):
    page, inside = [], False
    for line in iterator:
        if "<page>" in line:
            page = [line]
            inside = True
        elif "</page>" in line and inside:
            page.append(line)
            yield "\n".join(page)
            inside = False
            page = []
        elif inside:
            page.append(line)

def parse_page(xml_text):
    try:
        title_match = re.search(r"<title>(.*?)</title>", xml_text, re.DOTALL)
        text_match  = re.search(r"<text.*?>(.*?)</text>", xml_text, re.DOTALL)
        title = title_match.group(1).strip() if title_match else ""
        text  = text_match.group(1).strip() if text_match else ""
        return (title, text)
    except:
        return ("", "")

def is_dino_page(row):
    try:
        title = normalize_text(row[0])

        parts = title.split("-")

        for n in (1, 2, 3):
            if len(parts) >= n:
                candidate = "-".join(parts[:n])
                if candidate in dino_set:
                    return True

        return False

    except Exception:
        return False

schema = StructType([
    StructField("title", StringType(), True),
    StructField("text",  StringType(), True)
])

In [8]:
batch_size = 6_000_000
lines_buffer = []
processed_bytes = 0
total_size = 100 * 1024**3
start_time = time.time()
part = 0

with bz2.BZ2File(wiki_xml, 'rb') as infile, \
     tqdm(total=total_size, unit='B', unit_scale=True) as pbar:

    for raw_line in infile:
        processed_bytes += len(raw_line)
        pbar.update(len(raw_line))
        line = raw_line.decode("utf-8", errors="ignore")
        lines_buffer.append(line)

        if len(lines_buffer) >= batch_size:
            rdd = spark.sparkContext.parallelize(lines_buffer, 32)
            pages_rdd   = rdd.mapPartitions(group_pages)
            parsed_rdd  = pages_rdd.map(parse_page).filter(lambda x: x[0] != "")
            filtered_rdd = parsed_rdd.filter(is_dino_page)

            batch_df = spark.createDataFrame(filtered_rdd, schema)
            batch_df.write.mode("append").parquet(output_dir)

            lines_buffer = []
            del batch_df, filtered_rdd, parsed_rdd, pages_rdd, rdd
            gc.collect()
            spark.catalog.clearCache()

            part += 1
            elapsed = (time.time() - start_time) / 60
            print(f"Saved batch {part}, elapsed {elapsed:.1f} min")

    if lines_buffer:
        rdd = spark.sparkContext.parallelize(lines_buffer, 32)
        pages_rdd   = rdd.mapPartitions(group_pages)
        parsed_rdd  = pages_rdd.map(parse_page).filter(lambda x: x[0] != "")
        filtered_rdd = parsed_rdd.filter(is_dino_page)

        batch_df = spark.createDataFrame(filtered_rdd, schema)
        batch_df.write.mode("append").parquet(output_dir)

spark.stop()
print(f"\nAll dino pages saved to: {output_dir}")


Starting streaming extraction...


  1%|          | 951M/107G [00:30<11:45:17, 2.51MB/s]

Saved batch 1, elapsed 0.5 min


  2%|▏         | 1.77G/107G [00:53<6:39:03, 4.41MB/s]

Saved batch 2, elapsed 0.9 min


...

Saved batch 264, elapsed 53.7 min


112GB [53:51, 7.63MB/s]

Saved batch 265, elapsed 53.9 min


112GB [54:03, 34.6MB/s]



All dino pages saved to: data/output_clean


In [7]:
df = spark.read.parquet("filtered_dino_2")

print("Rows:", df.count())

Rows: 5956


In [8]:
from pyspark.sql import functions as F

df = df.withColumn(
    "taxon",
    F.when(
        F.regexp_extract("text", r"\|\s*taxon\s*=\s*([^\n]+)", 1) != "",
        F.regexp_extract("text", r"\|\s*taxon\s*=\s*([^\n]+)", 1)
    ).otherwise(None)
)

df = df.withColumn(
    "fossil_range",
    F.when(
        F.regexp_extract("text", r"\|\s*fossil_range\s*=\s*([^\n]+)", 1) != "",
        F.regexp_extract("text", r"\|\s*fossil_range\s*=\s*([^\n]+)", 1)
    ).otherwise(None)
)

df = df.withColumn(
    "description",
    F.when(
        F.regexp_extract("text", r"==Description==\s*\n\n([\s\S]*?)\n\n(?===|\Z)", 1) != "",
        F.regexp_extract("text", r"==Description==\s*\n\n([\s\S]*?)\n\n(?===|\Z)", 1)
    ).otherwise(None)
)

df = df.withColumn(
    "discovery",
    F.when(
        F.regexp_extract("text", r"==Discovery==\s*\n\n([\s\S]*?)\n\n(?===|\Z)", 1) != "",
        F.regexp_extract("text", r"==Discovery==\s*\n\n([\s\S]*?)\n\n(?===|\Z)", 1)
    ).otherwise(None)
)

df = df.withColumn(
    "history_of_discovery",
    F.when(
        F.regexp_extract("text", r"==History of discovery==\s*\n\n([\s\S]*?)\n\n(?===|\Z)", 1) != "",
        F.regexp_extract("text", r"==History of discovery==\s*\n\n([\s\S]*?)\n\n(?===|\Z)", 1)
    ).otherwise(None)
)

df = df.withColumn(
    "paleobiology",
    F.when(
        F.regexp_extract("text", r"==Paleobiology==\s*\n\n([\s\S]*?)\n\n(?===|\Z)", 1) != "",
        F.regexp_extract("text", r"==Paleobiology==\s*\n\n([\s\S]*?)\n\n(?===|\Z)", 1)
    ).otherwise(None)
)

df = df.withColumn(
    "paleoenvironment",
    F.when(
        F.regexp_extract("text", r"==Paleoenvironment==\s*\n\n([\s\S]*?)\n\n(?===|\Z)", 1) != "",
        F.regexp_extract("text", r"==Paleoenvironment==\s*\n\n([\s\S]*?)\n\n(?===|\Z)", 1)
    ).otherwise(None)
)

df = df.withColumn(
    "classification",
    F.when(
        F.regexp_extract("text", r"==Classification==\s*\n\n([\s\S]*?)\n\n(?===|\Z)", 1) != "",
        F.regexp_extract("text", r"==Classification==\s*\n\n([\s\S]*?)\n\n(?===|\Z)", 1)
    ).otherwise(None)
)

df = df.withColumn("title", F.regexp_replace("title", r"\t", " "))
df = df.withColumn("taxon", F.regexp_replace("taxon", r"\t", " "))
df = df.withColumn("fossil_range", F.regexp_replace("fossil_range", r"\t", " "))
df = df.withColumn("description", F.regexp_replace("description", r"\t", " "))
df = df.withColumn("discovery", F.regexp_replace("discovery", r"\t", " "))
df = df.withColumn("history_of_discovery", F.regexp_replace("history_of_discovery", r"\t", " "))
df = df.withColumn("paleobiology", F.regexp_replace("paleobiology", r"\t", " "))
df = df.withColumn("paleoenvironment", F.regexp_replace("paleoenvironment", r"\t", " "))
df = df.withColumn("classification", F.regexp_replace("classification", r"\t", " "))

(
    df.select("title", "taxon", "fossil_range", "description", "discovery", "history_of_discovery", "paleobiology", "paleoenvironment", "classification")
      .write
      .mode("overwrite")
      .option("header", "true")
      .option("delimiter", "\t")
      .csv("dino_extracted")
)

In [9]:
import pandas as pd
from pathlib import Path

def combine_tsv_parts(input_dir, output_file):
    input_path = Path(input_dir)
    part_files = sorted([f for f in input_path.glob("part-*") if f.is_file()])
    
    if not part_files:
        raise FileNotFoundError(f"No part files found in {input_dir}")
    
    dfs = []
    
    for part_file in part_files:
        try:
            df = pd.read_csv(part_file, delimiter='\t', dtype=str)
            dfs.append(df)
            print(f"Read {len(df)} rows from {part_file.name}")
        except Exception as e:
            print(f"Error reading {part_file.name}: {e}")
    
    combined_df = pd.concat(dfs, ignore_index=True)
    print(f"\nTotal rows before deduplication: {len(combined_df)}")
    
    combined_df = combined_df.drop_duplicates()
    print(f"Total rows after deduplication: {len(combined_df)}")
    
    combined_df.to_csv(output_file, sep='\t', index=False, encoding='utf-8')
    print(f"Wrote {len(combined_df)} data rows to: {output_file}")

combine_tsv_parts("dino_extracted", "dino_combined_2.tsv")

Read 1132 rows from part-00000-cca02c9e-d670-48b8-8c7b-aa1a8aa1739e-c000.csv
Read 818 rows from part-00001-cca02c9e-d670-48b8-8c7b-aa1a8aa1739e-c000.csv
Read 677 rows from part-00002-cca02c9e-d670-48b8-8c7b-aa1a8aa1739e-c000.csv
Read 458 rows from part-00003-cca02c9e-d670-48b8-8c7b-aa1a8aa1739e-c000.csv
Read 389 rows from part-00004-cca02c9e-d670-48b8-8c7b-aa1a8aa1739e-c000.csv
Read 365 rows from part-00005-cca02c9e-d670-48b8-8c7b-aa1a8aa1739e-c000.csv
Read 286 rows from part-00006-cca02c9e-d670-48b8-8c7b-aa1a8aa1739e-c000.csv
Read 310 rows from part-00007-cca02c9e-d670-48b8-8c7b-aa1a8aa1739e-c000.csv
Read 536 rows from part-00008-cca02c9e-d670-48b8-8c7b-aa1a8aa1739e-c000.csv
Read 275 rows from part-00009-cca02c9e-d670-48b8-8c7b-aa1a8aa1739e-c000.csv
Read 216 rows from part-00010-cca02c9e-d670-48b8-8c7b-aa1a8aa1739e-c000.csv
Read 166 rows from part-00011-cca02c9e-d670-48b8-8c7b-aa1a8aa1739e-c000.csv
Read 145 rows from part-00012-cca02c9e-d670-48b8-8c7b-aa1a8aa1739e-c000.csv
Read 116 ro

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, trim, concat_ws

spark = SparkSession.builder.appName("JoinDinosaurData").getOrCreate()

unified_df = spark.read.option("header", "true").option("delimiter", "\t").csv("unified_data.tsv")
dino_cleaned_df = spark.read.option("header", "true").option("delimiter", "\t").csv("dino_cleaned.tsv")

print(f"Old data rows: {unified_df.count()}")
print(f"Wiki data rows: {dino_cleaned_df.count()}")

unified_df = unified_df.dropDuplicates(["name"])
dino_cleaned_df = dino_cleaned_df.dropDuplicates(["title"])

unified_df = unified_df.withColumn(
    "join_key",
    lower(trim(col("name")))
)

dino_cleaned_df = dino_cleaned_df.withColumn(
    "join_key",
    lower(trim(col("title")))
)

for col_name in dino_cleaned_df.columns:
    if col_name not in ["join_key", "title"]:
        dino_cleaned_df = dino_cleaned_df.withColumnRenamed(col_name, col_name + "_wiki")

result_df = unified_df.join(
    dino_cleaned_df,
    on="join_key",
    how="left"
)

print(f"Result rows after join: {result_df.count()}")

matched_rows = result_df.filter(col("title").isNotNull()).count()
unmatched_unified_rows = result_df.filter(col("title").isNull()).count()

print(f"\n=== JOIN STATISTICS ===")
print(f"Rows from old data with matches in wiki data: {matched_rows}")
print(f"Rows from old data with NO matches in wiki data: {unmatched_unified_rows}")
print(f"Percentage of old data with a match from wiki data: {(matched_rows / unified_df.count() * 100):.2f}%")

result_df.drop("join_key").coalesce(1).write.mode("overwrite").option("header", "true").option("delimiter", "\t").csv("dino_joined_output")

import os
import shutil

output_dir = "dino_joined_output"
for filename in os.listdir(output_dir):
    if filename.startswith("part-"):
        shutil.move(os.path.join(output_dir, filename), "dino_joined_output.tsv")
        break

shutil.rmtree(output_dir, ignore_errors=True)

Old data rows: 6979
Wiki data rows: 5956
Result rows after join: 6990

=== JOIN STATISTICS ===
Rows from old data with matches in wiki data: 5888
Rows from old data with NO matches in wiki data: 1102
Percentage of old data with a match from wiki data: 84.62%
